# EfficientNet model training



In [ ]:
# EfficientNet model training

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import timm
import json
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print("🚀 EfficientNet-B3 Training Pipeline")
print("=" * 40)

# Check device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔧 Device: {device}")

# Load configuration
processed_root = Path("../../data/processed")
results_dir = Path("../../results")

with open(processed_root / "training_config.json", "r") as f:
    config = json.load(f)

class_weights = torch.load(processed_root / "fast_class_weights.pt", map_location='cpu')

print(f"📋 Training Configuration:")
print(f"   Classes: {config['num_classes']}")
print(f"   Class names: {config['class_names']}")
print(f"   Image size: {config['image_size']}")
print(f"   Device: {device}")

# Recreate the fast dataset (we need to reload it)
import xml.etree.ElementTree as ET
from collections import Counter
from PIL import Image

# Load simplified mappings
with open(results_dir / "simplified_class_mapping.json", "r") as f:
    class_mapping = json.load(f)

with open(results_dir / "detailed_to_simplified_mapping.json", "r") as f:
    detailed_to_simplified = json.load(f)

class FastVehicleDataset:
    """Lightweight dataset for training"""
    
    def __init__(self, split_name, max_files=100, transform=None):
        self.transform = transform
        self.data = self._load_data(split_name, max_files)
        print(f"📦 {split_name} dataset: {len(self.data)} samples")
    
    def _load_data(self, split_name, max_files):
        data_root = Path("../../data/raw")
        annos_dir = data_root / split_name / "annos"
        images_dir = data_root / split_name / "images"
        
        xml_files = list(annos_dir.glob("*.xml"))[:max_files]
        data_samples = []
        
        for xml_file in xml_files:
            try:
                tree = ET.parse(xml_file)
                root = tree.getroot()
                
                img_id = xml_file.stem
                img_path = images_dir / f"{img_id}.jpg"
                
                if not img_path.exists():
                    continue
                
                for obj in root.findall('object'):
                    try:
                        name_elem = obj.find('name')
                        if name_elem is None:
                            continue
                        
                        detailed_class = name_elem.text.strip().lower()
                        simplified_class = detailed_to_simplified.get(detailed_class, 'unknown')
                        
                        if simplified_class == 'unknown':
                            continue
                        
                        bbox_elem = obj.find('bndbox')
                        if bbox_elem is None:
                            continue
                        
                        xmin = max(0, int(float(bbox_elem.find('xmin').text)))
                        ymin = max(0, int(float(bbox_elem.find('ymin').text)))
                        xmax = int(float(bbox_elem.find('xmax').text))
                        ymax = int(float(bbox_elem.find('ymax').text))
                        
                        if xmax > xmin + 30 and ymax > ymin + 30:
                            data_samples.append({
                                'img_path': str(img_path),
                                'bbox': [xmin, ymin, xmax, ymax],
                                'class_idx': class_mapping[simplified_class],
                                'class_name': simplified_class
                            })
                    except:
                        continue
            except:
                continue
        
        return data_samples
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        try:
            item = self.data[idx]
            image = Image.open(item['img_path']).convert('RGB')
            bbox = item['bbox']
            cropped = image.crop(bbox)
            
            if self.transform:
                cropped = self.transform(cropped)
            
            return cropped, item['class_idx']
        except:
            # Return dummy data if error
            dummy_img = torch.zeros(3, 224, 224)
            return dummy_img, 0

# Define transforms for training
def get_training_transforms():
    return transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.RandomCrop((224, 224)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(degrees=10),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

def get_val_transforms():
    return transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

# Create datasets
print(f"\n📊 CREATING TRAINING DATASETS")
print("=" * 32)

train_dataset = FastVehicleDataset('train', max_files=100, transform=get_training_transforms())
val_dataset = FastVehicleDataset('val', max_files=50, transform=get_val_transforms())

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=0)

print(f"🔄 Data loaders ready:")
print(f"   Train batches: {len(train_loader)}")
print(f"   Val batches: {len(val_loader)}")

# Create EfficientNet model
print(f"\n🏗️ CREATING EFFICIENTNET-B3 MODEL")
print("=" * 35)

class EfficientNetClassifier(nn.Module):
    def __init__(self, num_classes=6, pretrained=True):
        super().__init__()
        
        # Load pretrained EfficientNet-B3
        self.backbone = timm.create_model('efficientnet_b3', pretrained=pretrained)
        
        # Get number of features
        num_features = self.backbone.classifier.in_features
        
        # Replace classifier
        self.backbone.classifier = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(num_features, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x):
        return self.backbone(x)

# Initialize model
model = EfficientNetClassifier(num_classes=config['num_classes'])
model = model.to(device)

print(f"✅ EfficientNet-B3 model created")
print(f"   Parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"   Trainable: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

# Create the missing directories first
models_dir = Path("../../models/saved_models")
models_dir.mkdir(parents=True, exist_ok=True)

results_plots_dir = Path("../../results/plots")
results_plots_dir.mkdir(parents=True, exist_ok=True)

print(f"✅ Created directories:")
print(f"   {models_dir}")
print(f"   {results_plots_dir}")

# Setup training
print(f"\n⚙️ TRAINING SETUP")
print("=" * 20)

# Loss function with class weights
criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))

# Optimizer
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)

# Learning rate scheduler
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

print(f"✅ Training setup complete:")
print(f"   Loss: CrossEntropyLoss with class weights")
print(f"   Optimizer: AdamW (lr=0.001)")
print(f"   Scheduler: CosineAnnealingLR")

# Training function
def train_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with tqdm(train_loader, desc="Training", leave=False) as pbar:
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            # Update progress bar
            pbar.set_postfix({
                'Loss': f'{loss.item():.3f}',
                'Acc': f'{100.*correct/total:.1f}%'
            })
    
    return running_loss / len(train_loader), 100. * correct / total

# Validation function
def validate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    return running_loss / len(val_loader), 100. * correct / total

# Training loop
print(f"\n🏃 STARTING TRAINING")
print("=" * 25)

num_epochs = 10  # Start with fewer epochs for quick testing
best_acc = 0.0
train_losses = []
train_accs = []
val_losses = []
val_accs = []

start_time = time.time()

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    print("-" * 20)
    
    # Train
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    
    # Validate
    val_loss, val_acc = validate(model, val_loader, criterion, device)
    
    # Update scheduler
    scheduler.step()
    
    # Save metrics
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    
    # Print results
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
    print(f"LR: {scheduler.get_last_lr()[0]:.6f}")
    
    # Save best model with proper path
    if val_acc > best_acc:
        best_acc = val_acc
        model_save_path = models_dir / "efficientnet_best.pth"
        torch.save(model.state_dict(), model_save_path)
        print(f"✅ New best model saved! (Val Acc: {val_acc:.2f}%)")

training_time = time.time() - start_time

print(f"\n🏁 TRAINING COMPLETE!")
print("=" * 25)
print(f"⏱️ Training time: {training_time:.2f}s")
print(f"🎯 Best validation accuracy: {best_acc:.2f}%")

# Plot training curves
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss', color='blue')
plt.plot(val_losses, label='Val Loss', color='red')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(train_accs, label='Train Acc', color='blue')
plt.plot(val_accs, label='Val Acc', color='red')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig(results_plots_dir / "efficientnet_training_curves.png", dpi=150, bbox_inches='tight')
plt.show()

# Save training results
training_results = {
    'model': 'EfficientNet-B3',
    'best_val_accuracy': float(best_acc),
    'final_train_accuracy': float(train_accs[-1]),
    'final_val_accuracy': float(val_accs[-1]),
    'training_time': float(training_time),
    'epochs': num_epochs,
    'train_losses': train_losses,
    'train_accs': train_accs,
    'val_losses': val_losses,
    'val_accs': val_accs,
    'parameters': sum(p.numel() for p in model.parameters()),
    'model_size_mb': sum(p.numel() * 4 for p in model.parameters()) / (1024 * 1024)  # Approximate MB
}

with open(results_dir / "efficientnet_training_results.json", "w") as f:
    json.dump(training_results, f, indent=2)

print(f"\n💾 RESULTS SAVED:")
print(f"   Model weights: {model_save_path}")
print(f"   Training curves: {results_plots_dir}/efficientnet_training_curves.png")
print(f"   Results: {results_dir}/efficientnet_training_results.json")

print(f"\n📊 FINAL RESULTS:")
print(f"   🎯 Best Val Accuracy: {best_acc:.2f}%")
print(f"   📈 Final Train Accuracy: {train_accs[-1]:.2f}%")
print(f"   ⏱️ Training Time: {training_time:.1f}s")
print(f"   🏋️ Model Size: {training_results['model_size_mb']:.1f}MB")

print(f"\n🚀 EFFICIENTNET TRAINING COMPLETE!")
print("Next: Train ResNet with Attention mechanism")

🚀 EfficientNet-B3 Training Pipeline
🔧 Device: cpu
📋 Training Configuration:
   Classes: 6
   Class names: ['auto_rickshaw', 'bus', 'car', 'motorcycle', 'scooter', 'truck']
   Image size: 224
   Device: cpu

📊 CREATING TRAINING DATASETS
📦 train dataset: 414 samples
📦 val dataset: 203 samples
🔄 Data loaders ready:
   Train batches: 52
   Val batches: 26

🏗️ CREATING EFFICIENTNET-B3 MODEL
✅ EfficientNet-B3 model created
   Parameters: 11,091,246
   Trainable: 11,091,246
✅ Created directories:
   ..\..\models\saved_models
   ..\..\results\plots

⚙️ TRAINING SETUP
✅ Training setup complete:
   Loss: CrossEntropyLoss with class weights
   Optimizer: AdamW (lr=0.001)
   Scheduler: CosineAnnealingLR

🏃 STARTING TRAINING

Epoch 1/10
--------------------


Train Loss: 1.2396, Train Acc: 55.80%
Val Loss: 1.3076, Val Acc: 73.40%
LR: 0.000976
✅ New best model saved! (Val Acc: 73.40%)

Epoch 2/10
--------------------


Train Loss: 0.9984, Train Acc: 69.57%
Val Loss: 0.9580, Val Acc: 75.37%
LR: 0.000905
✅ New best model saved! (Val Acc: 75.37%)

Epoch 3/10
--------------------


Train Loss: 0.6221, Train Acc: 79.47%
Val Loss: 1.0038, Val Acc: 79.80%
LR: 0.000794
✅ New best model saved! (Val Acc: 79.80%)

Epoch 4/10
--------------------


Train Loss: 0.6173, Train Acc: 79.47%
Val Loss: 0.5418, Val Acc: 82.27%
LR: 0.000655
✅ New best model saved! (Val Acc: 82.27%)

Epoch 5/10
--------------------


...

Train Loss: 0.2822, Train Acc: 90.58%
Val Loss: 0.5405, Val Acc: 85.22%
LR: 0.000345
✅ New best model saved! (Val Acc: 85.22%)

Epoch 7/10
--------------------


Train Loss: 0.1749, Train Acc: 93.00%
Val Loss: 0.5056, Val Acc: 87.19%
LR: 0.000206
✅ New best model saved! (Val Acc: 87.19%)

Epoch 8/10
--------------------


Train Loss: 0.1079, Train Acc: 95.89%
Val Loss: 0.4418, Val Acc: 88.67%
LR: 0.000095
✅ New best model saved! (Val Acc: 88.67%)

Epoch 9/10
--------------------


Train Loss: 0.0932, Train Acc: 97.10%
Val Loss: 0.4403, Val Acc: 87.68%
LR: 0.000024

Epoch 10/10
--------------------


Train Loss: 0.0680, Train Acc: 98.31%
Val Loss: 0.4558, Val Acc: 87.68%
LR: 0.000000

🏁 TRAINING COMPLETE!
⏱️ Training time: 1070.18s
🎯 Best validation accuracy: 88.67%
